In [5]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import math
from nltk.tokenize import word_tokenize

quotes = {}
numquotes = 0


url = 'https://quotes.toscrape.com' # 'https://quotes.toscrape.com/page/2/'
soup = BeautifulSoup(urlopen(url), 'html.parser')

texts = soup.find_all("div", class_="quote")

for text in texts:
    numquotes = numquotes + 1
    quote = text.find("span", class_="text").text
    #print(quote)
    
    author = text.find("small", class_="author").text
    if author in quotes:
        quotes[author].append(quote[1:len(quote) - 1])
    else:
        quotes[author] = [quote[1:len(quote) - 1]]
#print(soup.prettify())

for i in range(9):
    url = 'https://quotes.toscrape.com/page/' + str(i + 2) + '/'
    
    #print(url)
    soup = BeautifulSoup(urlopen(url), 'html.parser')
    
    texts = soup.find_all("div", class_="quote")

    for text in texts:
        numquotes = numquotes + 1
        quote = text.find("span", class_="text").text
        author = text.find("small", class_="author").text
        if author in quotes:
            quotes[author].append(quote[1:len(quote) - 1])
        else:
            quotes[author] = [quote[1:len(quote) - 1]]

#for author in quotes:
#    print(author)
#    for quote in quotes[author]:
#        print('\t' + quote)


tf = {}
for author in quotes:
    for quote in quotes[author]:
        term_frequency = {}

        max = 0

        for word in word_tokenize(quote):
            if word in term_frequency:
                term_frequency[word] = term_frequency[word] + 1
                if max < term_frequency[word]:
                    max = term_frequency[word]
            else:
                term_frequency[word] = 1
                if max < 1:
                    max = 1

        for token in term_frequency:
            term_frequency[token] = term_frequency[token] / max

        tf[quote] = term_frequency

# find document frequency

df = {}
for quote in tf:
    for token in tf[quote]:
        if token in df:
            df[token] = df[token] + 1
        else:
            df[token] = 1
    #print(tf[file])

# find tf-idf for each token

tf_idf = {}

for quote in tf:
    #print(quote)
    tf_idf[quote] = {}
    for token in tf[quote]:
        _tf = tf[quote][token]
        
        _idf = math.log(numquotes / (df[token] + 1))
        #print("    " + token + ": ")
        #print("       tf: " + str(tf[quote][token]) + " | df: " + str(df[token]) + " | tf-idf: " + str(_tf * _idf))
        tf_idf[quote][token] = _tf * _idf
    #print()
    #print()

file = open("tf_idf.csv", 'w', encoding="utf-8")
    
for quote in tf_idf:
    for token in tf_idf[quote]:
        file.write(quote.replace(',', '=='))
        file.write(', ')
        file.write(token.replace(',', '=='))
        file.write(', ')
        file.write(str(tf_idf[quote][token]))
        file.write('\n')
file.close()
print("done")

done
